In [ ]:
from federated import agglomerate
import torch
from time import sleep
from server import move_left, move_forward, move_right, stop_robot
from server import display_camera_stream
from server import get_state, set_drive, get_weights, set_weights

In [ ]:
def action(box_cords):
    if box_cords < 0:
        return (0, -0.2)
    pass

def PSO(robots: list[list[(float, float)]]):
    best_score = (-1, -1)
    best_robot = 0
    acts = [(0,0)]*robots
    for index, robot in enumerate(robots): 
        box_cords = robot[0]
        if box_cords > 0:
            score = abs((0.5, 0.5) - box_cords)
        else:
            score = (-1, -1)
        acts[index] = action(score)
        
        if score > best_score:
            best_robot = index
    
    if len(robots) > 1 and best_score > 0:
        for index, robot in enumerate(robots): 
            if index == best_robot:
                continue
            acts[index] *= (sum(best_score)/2) * robots[best_robot]

    return acts

In [ ]:
IPS = ["194.47.156.140"]
COLOURS = ["GREEN"]
weights = [None]*len(IPS)
states = [(0,0)]*len(IPS)
for i in range(0, 100):
    states = []
    for index, IP in enumerate(IPS):
        states.append(tuple(get_state(IP).values()))

    #descisions = PSO(states)

    for index, IP in enumerate(IPS):
        #move_robot =  set_drive(IP, descisions[index][0], descisions[index][1])
        set_drive(IP, states[index][0], states[index][1]+0.01)

    sleep(1) 

    for index, IP in enumerate(IPS):
        stop_robot(IP)
    
    if i % 10 == 0:
        for index, IP in enumerate(IPS):
            weights[index] = get_weights(IP)

        agg_weight = agglomerate(weights)
        
        for index, IP in enumerate(IPS):
            send_weights = set_weights(IP, agg_weight)
            pass

for index, IP in enumerate(IPS):
    stop_robot = stop_robot(IP)
    weights[index] = get_weights(IP)

agg_weight = agglomerate(weights)
torch.save(agg_weight, "model.pth")